# **Data Cleaning Project**
This project is done for the learning purpose. It will give u the practical understanding of data cleaning

## 1. Import Librabries & Load Data

In [178]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [179]:
# Load dataset
file_path = "../data/data.csv"
df = pd.read_csv(file_path)
df_raw = df.copy # Copy of the raw dataset

## 2. Initial Profiling & Inspection

In [180]:
# Number of rows and columns
print(f"{df.shape[0]} rows and {df.shape[1]} columns.")

90 rows and 8 columns.


In [181]:
# Get first few rows of data to get overview
df.head()

,OrderID,Date,CustomerName,Product,Quantity,Price,Total,Region
0,1001,1/5/2021,John Smith,Laptop,1,"$1,200",1200,North
1,1002,1/7/2021,Jane Doe,Mobile,2,300,600,South
2,1003,1/8/2021,Michael Brown,Laptop,,$1,200,NaN
3,1004,2021-13-09,Sarah Miller,Tablet,3,$250,750,West
4,1005,1/10/2021,John Smith,Laptop,1,1200,1200,North


In [182]:
# Get types of data in each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   OrderID       90 non-null     int64 
 1   Date          87 non-null     object
 2   CustomerName  85 non-null     object
 3   Product       87 non-null     object
 4   Quantity      85 non-null     object
 5   Price         89 non-null     object
 6   Total         85 non-null     object
 7   Region        88 non-null     object
dtypes: int64(1), object(7)
memory usage: 5.8+ KB


In [183]:
# Get initial summary with basic statistics
df.describe(include="all")

,OrderID,Date,CustomerName,Product,Quantity,Price,Total,Region
count,90.000000,87,85,87,85,89,85,88
unique,NaN,86,10,3,12,7,16,7
top,NaN,1/12/2021,Adam Lee,Laptop,1,"$1,200",300,West
freq,NaN,2,17,32,36,22,23,32
mean,1045.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,26.124701,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1001.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1023.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1045.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1067.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [184]:
# Get missing value summary
print("\nNumber of missing values in each column")
df.isnull().sum().sort_values(ascending=False)


Number of missing values in each column


CustomerName    5
Quantity        5
Total           5
Date            3
Product         3
Region          2
Price           1
OrderID         0
dtype: int64

In [185]:
# Get duplicated value summary
print(f"Number of duplicated rows is {df.duplicated().sum()}")

Number of duplicated rows is 0


## 3. Fixing Data Types

In [186]:
# Convert "Date" column data type into date type
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

In [187]:
# Fix inconsistent formatting of "Price" column
df["Price"] = df["Price"].str.replace("$", "").str.replace(",", "")

In [188]:
# Convert "Price, Quantity, Total" column into numerical datatype
lists = ["Price", "Quantity", "Total"]
for col in lists:
    df[col] = pd.to_numeric(df[col], errors="coerce")

In [189]:
# Convert "OrderID" column into object data type
df["OrderID"] = df["OrderID"].astype("object")

## 4. Text Normalization / Categorical Cleaning

In [190]:
# Correct inconconsistent naming format
cleaned_name = df["CustomerName"].str.replace(r"\s+", " ", regex=True).str.strip().str.title()
df["CustomerName"] = cleaned_name

In [191]:
# Correct negative quanity number
df["Quantity"] = np.abs(df["Quantity"])

## 4. Missing Values Handling

In [192]:
# Impute date column with forward fill technique as it is date data type
df["Date"] = df["Date"].ffill()

In [193]:
# Impute customer name with unknown value as we don't know the name
df["CustomerName"] = df["CustomerName"].fillna("Unknown")

In [194]:
# Impute product name with mode value as it is categorical value
df["Product"] = df["Product"].fillna(df["Product"].mode()[0])

In [195]:
# Impute quantity with median as it has outlier
df["Quantity"].fillna(df["Quantity"].median(), inplace=True)
df[df["Quantity"] == 0] = df["Quantity"].mode()[0]

C:\Users\syful\AppData\Local\Temp\ipykernel_4740\3066884759.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Quantity"].fillna(df["Quantity"].median(), inplace=True)
C:\Users\syful\AppData\Local\Temp\ipykernel_4740\3066884759.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df[df["Quantity"] == 0] = df["Quantity"].mode()[0]


In [196]:
# Impute price with median as it has outlier also
df["Price"] = df["Price"].fillna(df["Price"].median())
df[df["Price"] == 1] = df["Price"].median()

In [197]:
# Impute total multiplying price with quantity
df["Total"] = df["Price"] * df["Quantity"]

In [198]:
# Impute region with mode as it is categorical value
df["Region"] = df["Region"].fillna(df["Region"].mode()[0])

# Inconsistent region name with mode value
df["Region"] = df["Region"].str.replace("2400", df["Region"].mode()[0])
df["Region"] = df["Region"].str.replace("1200", df["Region"].mode()[0])
df["Region"] = df["Region"].str.replace("240000", df["Region"].mode()[0])

In [199]:
df["Region"] = df["Region"].str.replace("West00", "West")

## 5. Outlier Detection & Treatment

In [200]:
# Remove outliers of numeric data type
def remove_outliers(col, df):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    high_range = q3 + (1.5 * iqr)
    low_range = q1 - (1.5 * iqr)
    return df[(df[col] < high_range) & (df[col] > low_range)]

for col in df.select_dtypes(include=["int", "float"]).columns:
    df = remove_outliers(col, df)

## 6. Duplicate Handling

In [202]:
# Handling duplacted row
df_before = df.shape
df = df.drop_duplicates()
df_after = df.shape
df_before, df_after

((80, 8), (80, 8))

## 7. Validation Checks

In [204]:
print("Final rows:", df.shape[0])
print("Final columns:", df.shape[1])
df.describe(include="all")

Final rows: 80
Final columns: 8


C:\Users\syful\AppData\Local\Temp\ipykernel_4740\616935764.py:3: FutureWarning: The behavior of value_counts with object-dtype is deprecated. In a future version, this will *not* perform dtype inference on the resulting index. To retain the old behavior, use `result.index = result.index.infer_objects()`
  df.describe(include="all")


,OrderID,Date,CustomerName,Product,Quantity,Price,Total,Region
count,80.0,80,80,80,80.000000,80.000000,80.00000,80
unique,80.0,76,6,3,NaN,NaN,NaN,4
top,1001.0,2021-01-19 00:00:00,John Smith,Mobile,NaN,NaN,NaN,West
freq,1.0,2,17,29,NaN,NaN,NaN,32
mean,NaN,NaN,NaN,NaN,1.912500,554.375000,943.12500,NaN
std,NaN,NaN,NaN,NaN,0.970368,425.841962,714.66305,NaN
min,NaN,NaN,NaN,NaN,1.000000,250.000000,300.00000,NaN
25%,NaN,NaN,NaN,NaN,1.000000,250.000000,300.00000,NaN
50%,NaN,NaN,NaN,NaN,2.000000,300.000000,750.00000,NaN
75%,NaN,NaN,NaN,NaN,3.000000,1200.000000,1200.00000,NaN


# 8. Export Cleaned Dataset

In [205]:
df.to_csv("../data/cleaned_data.csv", index=False)
df.head()

,OrderID,Date,CustomerName,Product,Quantity,Price,Total,Region
0,1001,2021-01-05 00:00:00,John Smith,Laptop,1.0,1200.0,1200.0,North
1,1002,2021-01-07 00:00:00,Jane Doe,Mobile,2.0,300.0,600.0,South
3,1004,2021-01-08 00:00:00,Sarah Miller,Tablet,3.0,250.0,750.0,West
4,1005,2021-01-10 00:00:00,John Smith,Laptop,1.0,1200.0,1200.0,North
5,1006,2021-01-11 00:00:00,Unknown,Mobile,1.0,300.0,300.0,South
